In [61]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')
movies = movies.merge(credits,on = "title")

Columns Selection


In [62]:
''' genre, id(movie poster will be fetched this way), keywords, (since english movies are very much so language is not a contributing factor), title, overview, 
cast, crew [director], '''

' genre, id(movie poster will be fetched this way), keywords, (since english movies are very much so language is not a contributing factor), title, overview, \ncast, crew [director], '

Selecting useful data only

In [63]:
movies = movies[['movie_id','title','cast','genres','overview','crew','keywords']]

Data Pre-Processing (Duplicate, missing data)

In [64]:
movies.isnull().sum()
movies.dropna(inplace=True) # removing blank data


In [65]:
#Duplicate Data
movies.duplicated().sum() # no duplicate data found


0

In [66]:
# Processing 'genre' column of movies
import ast
def convert(obj):
    l = []
    for i in ast.literal_eval(obj):
        l.append(i['name'])
    return l

In [67]:

movies['genres'] = movies['genres'].apply(convert)

In [68]:
movies['keywords'] = movies['keywords'].apply(convert)

In [69]:
import ast
def convert_cast(obj):
    l1 = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            l1.append(i['name'])
            counter+=1
        else:
            break
    return l1


In [70]:
# processing cast names
movies['cast'] = movies['cast'].apply(convert_cast)

In [71]:
# refining director name
import ast
def convert_director(obj):
    l2 = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            l2.append(i['name'])
            break
    return l2

In [72]:
movies['crew'] = movies['crew'].apply(convert_director)

In [73]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [74]:
# replacing " " by null
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])

In [75]:
# creating a common tag
movies['tags'] = movies['overview']+movies['cast']+movies['crew']+movies['genres']+movies['keywords']

In [76]:
newmovie = movies[['movie_id','title','tags']]

In [77]:
#convert list to string
newmovie['tags'] = newmovie['tags'].apply(lambda x:" ".join(x))

/var/folders/gb/z0fpl4gj7fz771p_z5xhdpph0000gn/T/ipykernel_12957/1689516966.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newmovie['tags'] = newmovie['tags'].apply(lambda x:" ".join(x))


In [78]:
newmovie['tags'] = newmovie['tags'].apply(lambda x:x.lower())

/var/folders/gb/z0fpl4gj7fz771p_z5xhdpph0000gn/T/ipykernel_12957/3207758352.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newmovie['tags'] = newmovie['tags'].apply(lambda x:x.lower())


In [79]:
# order of tag - 'cast'+'crew'+'genres'+'keywords'+'overview'.
# We will  find the similarity between movies using 'tag'.
# Bag of Words technique is used to find the similarity.

In [80]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000,stop_words = 'english')

In [81]:
vectors = cv.fit_transform(newmovie['tags']).toarray()

In [82]:
import sys
np.set_printoptions(threshold=sys.maxsize)

# Here we have many words which mean the same but have been used to give a dimension to the vector such as 'action' and 'actions', so we need to equalize
# them and make vector formation more effective, we will apply stemming using nltk library

In [83]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [84]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [85]:
newmovie['tags'] = newmovie['tags'].apply(stem)

# order of tag - 'overview'+'cast'+'crew'+'genres'+'keywords'

/var/folders/gb/z0fpl4gj7fz771p_z5xhdpph0000gn/T/ipykernel_12957/2430513417.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newmovie['tags'] = newmovie['tags'].apply(stem)


In [86]:
#finding the cosine distance(angular distance) to find the similarity
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vectors)


In [87]:
newmovie = newmovie.drop([4806,4807,4808])


In [88]:
#Finding the Movie
def recommend(movie):
    try:
        index1 = newmovie[newmovie['title'] == movie].index[0]
    except:
        index1 = -1
    if index1 == -1:
        print("Movie not in Database")
    elif index1 <=4806:
        distances = similarity[index1]
        movies_list = sorted(list(enumerate(distances)),reverse = True,key = lambda x:x[1])[1:10]
        for i in movies_list:
            print(newmovie['title'][i[0]])


In [89]:
#Enter the movie name:
# Example :
recommend("Dune")

Red Tails
Flyboys
Enemy at the Gates
Crazy/Beautiful
Summer Storm
The Lovers
Captain America: The First Avenger
Lawrence of Arabia
The Great Raid
